In [3]:
import torch

# Humandoid MARL
from Humanoid_MARL import envs
from Humanoid_MARL.envs.base_env import GymWrapper, VectorGymWrapper
from Humanoid_MARL.utils.torch_utils import save_models, load_models
from Humanoid_MARL.agent.ppo.train_torch import Agent, eval_unroll, get_agent_actions
from Humanoid_MARL.envs.torch_wrapper import TorchWrapper
from IPython.display import HTML, clear_output
from brax.io import html
import jax
from Humanoid_MARL import envs
from Humanoid_MARL.utils.utils import load_reward_config
from Humanoid_MARL import CONFIG_REWARD
import mediapy as media

In [4]:
config = {
        'num_timesteps': 100_000_000,
        'eval_frequency': 10,
        'episode_length': 1000,
        'unroll_length': 10,
        'num_minibatches': 32,
        'num_update_epochs': 8,
        'discounting': 0.97,
        'learning_rate': 3e-4,
        'entropy_cost': 1e-3,
        'num_envs': 2048,
        'batch_size': 512,
        'env_name': "humanoids",
        'render' : True,
        'device' : 'cuda',
        'model_path' : "../models/20240226_215943_ppo_humanoids_decent_chase.pt",
        'video_length' : 300,
        'device_idx': 1
    }


env_config = load_reward_config(CONFIG_REWARD, config['env_name'])
env = envs.create(
        config['env_name'],
        auto_reset=True,
        **env_config,
    )

env = GymWrapper(env, get_jax_state=True)
env = TorchWrapper(env, device=config['device'], get_jax_state=True)

# env warmup
observation = env.reset()
action = torch.zeros(env.action_space.shape[0] * env.num_agents).to(config['device'])
env.step(action)
agents = load_models(config['model_path'], Agent, device=config['device'])
jax_states = []
num_steps = 1000

eval_reward = 0.0
episodes = torch.zeros((), device = config['device'])
for i in range(num_steps):
    print(f"{i} / {num_steps}")
    logits, action = get_agent_actions(agents, observation, env.obs_dims)
    jax_state, observation, reward, done, info = env.step(Agent.dist_postprocess(action[0]))
    episodes += torch.sum(done)
    jax_states.append(jax_state)
    print(f"{i} | {info} | DONE [{done}] | Reward [{reward}]")
    eval_reward += reward
print(f"Total Reward | {eval_reward / episodes}")

Models loaded from ../models/20240226_215943_ppo_humanoids_decent_chase.pt
0 / 10
0 | {'distance_from_origin': tensor([0.9494, 1.7068], device='cuda:0'), 'first_obs': tensor([ 1.4084e+00,  1.0084e+00, -1.2134e-03,  2.9750e-03,  1.4056e-03,
        -4.4896e-03, -6.8397e-03,  8.2619e-03,  9.4048e-03,  2.4356e-03,
         9.4220e-03,  3.3471e-03, -1.3046e-03, -5.8691e-03,  1.6548e-03,
        -9.9999e-03,  1.7405e-03, -8.1614e-03, -3.8471e-03, -6.2480e-04,
         3.5855e-03,  3.3886e-03,  1.3921e+00,  9.9902e-01, -1.8133e-03,
         3.2461e-03, -2.4885e-03,  2.9256e-03,  6.5626e-03,  5.6901e-03,
        -6.5986e-03,  6.5693e-03, -7.1839e-03, -6.7874e-03,  5.2287e-03,
         2.5417e-03,  8.1917e-03, -2.0397e-03,  9.9297e-03,  9.2369e-03,
        -7.8070e-04, -6.7274e-03,  4.6432e-03,  5.3263e-03, -1.0590e-03,
         5.0822e-03,  7.3168e-03,  7.0360e-03, -8.5047e-03, -6.7260e-03,
         2.4848e-03,  7.9382e-03,  6.5685e-03,  4.6577e-03,  5.8189e-03,
         1.8253e-04,  2.3478e-

In [ ]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 

In [ ]:
HTML(html.render(env.sys, [jax_state.pipeline_state for jax_state in jax_states])) 

: 